In [72]:
@generated function harmonicMean(I::Vararg{Integer, N}) where N
    ex = :(1 / I[$1])
    for i = 2 : N
        ex = :( 1 / I[$i] + $ex)
    end
    return :(N / $ex)
end
    

harmonicMean (generic function with 1 method)

In [73]:
function harmonicMean_impl(I::Vararg{Integer, N}) where N
    ex = :(1 / I[$1])
    for i = 2 : N
        ex = :( 1 / I[$i] + $ex)
    end
    return :(N / $ex)
end

harmonicMean_impl (generic function with 1 method)

In [74]:
harmonicMean_impl(1, 4, 4)

:(N / (1 / I[3] + (1 / I[2] + 1 / I[1])))

In [75]:
harmonicMean(1,4,4)

2.0

Automatic differentiation

In [76]:
function autodiff(ex::Expr)::Expr
    if(length(ex.args) == 1) 
        return ex
    else
        args = ex.args
        autodiff(Val{args[1]}, args[2:end])
    end
end

autodiff (generic function with 7 methods)

In [77]:
function autodiff(x::T) where T <: Number
    return 0
end

autodiff (generic function with 7 methods)

In [78]:
function autodiff(x::Symbol) 
    return 1
end

autodiff (generic function with 7 methods)

In [79]:
function autodiff(::Type{Val{:+}}, I::Array{Any, 1}) 
    diff1 = autodiff(I[1])
    ex = :($diff1)
    
    for i = 2:length(I)
        diff = autodiff(I[i])
        ex = :($ex + $diff)
    end
    return ex
end

autodiff (generic function with 7 methods)

In [80]:
function autodiff(::Type{Val{:-}}, I::Array{Any, 1}) 
    diff1 = autodiff(I[1])
    ex = :($diff1)
    
    for i = 2:length(I)
        diff = autodiff(I[i])
        ex = :($ex - $diff)
    end
    return ex
end

autodiff (generic function with 7 methods)

In [81]:
function autodiff(::Type{Val{:*}}, I::Array{Any, 1}) 
    
    if(length(I) == 2)
        diff1 = autodiff(I[1])
        exdiff1 = :($diff1)
        ex1 = I[1]
    
        diff2 = autodiff(I[2])
        exdiff2 = :($diff2)
        ex2 = I[2]
        diff =  :($ex1 * $exdiff2 + $ex2 * $exdiff1)
        return diff
    else
        ex = I[2]
        for i = 3:length(I)
            tmp = I[i]
            ex = :($ex * $tmp)
        end
        
        ex1 = I[1]
        exdiff1 = autodiff(I[1])
        compound_diff = autodiff(ex)

        return :($exdiff1 * $ex + $ex1 * $compound_diff)
    end

end

autodiff (generic function with 7 methods)

In [82]:
function autodiff(::Type{Val{:/}}, I::Array{Any, 1}) 
    diff1 = autodiff(I[1])
    exdiff1 = :($diff1)
    ex1 = I[1]
    
    diff2 = autodiff(I[2])
    exdiff2 = :($diff2)
    ex2 = I[2]
 
    return :(($exdiff1 * $ex2 - $ex1 * $exdiff2) / ($ex2 * $ex2))
end

autodiff (generic function with 7 methods)

Examples:

In [83]:
ex = :(2)
eval(autodiff(ex))

0

In [84]:
ex = :(2x)
eval(autodiff(ex))

2

In [85]:
ex = :(x*x)
x = 1
eval(autodiff(ex))

2

In [86]:
ex = :(x + x*x*x)
x = 1
eval(autodiff(ex))

4

In [87]:
ex = :((x + 2) / (x - 4) * (7*x*x + 2))
x = 5
eval(autodiff(ex))

-572.0

http://www.wolframalpha.com/input/?i=((x+%2B+2)+%2F+(x+-+4)+*+(7*x*x+%2B+2))%27+,+x+%3D5

In [88]:
ex = :(14/(x * (x-7)) + 28x)
x = 5
eval(autodiff(ex))

27.58

http://www.wolframalpha.com/input/?i=(14%2F(x+*+(x-7))+%2B+28x)%27+,+x+%3D+5

In [90]:
ex = :(4 / (x+2) / (x+7) / (x * x))
x = 5
eval(autodiff(ex))

-0.0011927437641723355

http://www.wolframalpha.com/input/?i=(4+%2F+(x%2B2)+%2F+(x%2B7)+%2F+(x+*+x))%27+,+x+%3D5